# Failure probability calculations with FORM

In this example, we will demonstrate how to perform reliability calculations using the First Order Reliability Method (FORM).

First, let's import the necessary package:

In [58]:
from ptk import *

Next, we define a simple limit state function: 

$Z = 1.9 - (a+b)$

This is a linear model involving two variables, $a$ and $b$.

In [59]:
def linear(a, b):
    return 1.9 - (a+b)

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [60]:
project = ReliabilityProject()
project.model = linear

We assume that variables $a$ and $b$ are uniformly distributed over the interval $[-1, 1]$. This is defined as follows:

In [61]:
stochast1 = project.variables["a"]
stochast1.distribution = "uniform"
stochast1.minimum = -1
stochast1.maximum = 1

stochast2 = project.variables["b"]
stochast2.distribution = "uniform"
stochast2.minimum = -1
stochast2.maximum = 1

Next, we define the reliability method: `FORM`. 

[The possible methods are: form, numerical_integration, crude_monte_carlo, importance_sampling,adaptive_importance_sampling, subset_simulation.]

We also choose the calculation settings: `variation_coefficient` and `maximum_iterations`.

[The possible calculation settings are: variation_coefficient, relaxation_factor/loops, minimum/maximum_samples/iterations/directions/variance_loops, fraction_failed]

In [62]:
project.settings.reliability_method = "form"

project.settings.variation_coefficient = 0.02
project.settings.maximum_iterations = 20

We use `project.run()` to execute the reliability analysis:

In [63]:
project.run()

The results are witten to `project.design_point` and consist of:
* reliability index $\beta$
* failure probability $P_f$
* influence coefficients $\alpha$-values
* design point $x$-values
* information about the convergence of FORM


In [64]:
def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")

read_results(project.design_point)

Beta = 2.772351852475011
Probability of failure = 0.0027826422680776335
a: alpha = -0.7071067811865476, x = 0.9500449635563615
b: alpha = -0.7071067811865476, x = 0.9500449635563615
Converged (convergence = 0.004050836299007645 < 0.02)


In the above example, variables $a$ and $b$ are independent. However, it is also possible to introduce a correlation between $a$ and $b$. The example below demonstrates this with a partial correlation coefficient of $0.8$.

In [65]:
project.correlation_matrix[("a", "b")] = 0.8

project.run()

read_results(project.design_point)


Beta = 2.0665776344233597
Probability of failure = 0.01938698503961158
a: alpha = -0.9456009795757881, x = 0.9493174459367089
b: alpha = -0.3253287374722835, x = 0.950784099184725
Converged (convergence = 0.0038834889996542717 < 0.02)
